# Training a Phoneme Language Model #
In this notebook I train a language model over English sounds (also known as [phonemes](https://en.wikipedia.org/wiki/Phoneme)). The data for English pronunciations comes from the CMU Pronouncing Dictionary. The pronunciations in the pronouncing dictionary are in [ARPABET](https://en.wikipedia.org/wiki/ARPABET), a set of symbols representing English sounds. So in ARPABET "fish" is pronounced as /F IH1 SH/.

By training on tens of thousands of pronunciations the model will hopefully learn [English phonotactics](https://en.wikipedia.org/wiki/Phonotactics#English_phonotactics), the rules that govern what sounds like a valid English word. For example, /F AH1 N/ ("fun") sounds good, but /NG S ER1/ (maybe represented as "ngsr") does not.


Check out the notebook `Phoneme Exploration.ipynb` if you want to see the model used.

In [1]:
import torch
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import ParameterGrid
from torch.nn import functional as F
from tqdm.notebook import tqdm

from sonorous.languagemodel import LanguageModel, ModelParams, Vocabulary
from sonorous.pronunciationdata import load_pronunciations
from sonorous.utils import split_data

## The Data ##
The data for this model comes from the [CMU Pronouncing Dictionary](http://www.speech.cs.cmu.edu/cgi-bin/cmudict), which contains over one hundred thousand pronunciations. Each pronuncation is in [ARPABET](https://en.wikipedia.org/wiki/ARPABET), a set of symbols for representing English speech sounds. In ARPABET the word "fish" is represented by the sequence of phonemes /F IH1 SH/. You can probably guess the first and third sounds. The vowel in the middle has "1" at the end to indicate it has the primary stress.

I'll use the `load_pronunciations` function to load the Pronouncing Dictionary into a Pandas DataFrame.

In [2]:
pronunciations = load_pronunciations()
print(f"There are {len(pronunciations):,} pronunciations.")
print()
print("Sample of 5 pronunciations:")
pronunciations.sample(5)

There are 131,964 pronunciations.

Sample of 5 pronunciations:


,pronunciation,num_phonemes,num_syllables,num_primary_stressed_syllables
word,,,,
thickened,"(ˈ, θ, ɪ, k, ə, n, d)",6,2,1
orphanages,"(ˈ, ɔ, r, f, ə, n, ɪ, dʒ, ɪ, z)",9,4,1
discontinue,"(d, ɪ, s, k, ə, n, ˈ, t, ɪ, n, j, uː)",11,4,1
startup,"(ˈ, s, t, ɑː, r, ˌ, t, ə, p)",7,2,1
enduring,"(ɛ, n, d, ˈ, j, ʊ, r, ɪ, ŋ)",8,3,1


Here is the pronunciation for "fish":

In [3]:
pronunciations.loc['fish']

pronunciation                     (ˈ, f, ɪ, ʃ)
num_phonemes                                 3
num_syllables                                1
num_primary_stressed_syllables               1
Name: fish, dtype: object

And here are all of the pronunciations for the word "tomato":

In [4]:
pronunciations.loc['tomato']

,pronunciation,num_phonemes,num_syllables,num_primary_stressed_syllables
word,,,,
tomato,"(t, ə, ˈ, m, eɪ, ˌ, t, oʊ)",6,3,1
tomato,"(t, ə, ˈ, m, ɑː, ˌ, t, oʊ)",6,3,1


## The Model ##
The module `languagemodel.py` contains a class `LanguageModel` that implements a simple neural language model. It's a PyTorch neural network comprised of the following layers:
1. **Embedding layer** to translate each phoneme into a dense vector. Note that in the code this is called the _encoder since it encodes input phonemes into a representation the model can work with.
2. An recurrent neural network (**RNN**) layer that processes each input phoneme sequentially and for each step generates (a) a hidden representation to pass on to the next step and (b) an output.
3. A **linear layer** that decodes the outputes (2b) into distributions over each phoneme. Note that in the code this is called the _docoder since it decodes the model's internal representations back into phonemes.

Let's go through a simple example of what happens when we pass the pronunciation /F IH1 SH/ through the model. Ultimately what I want ouf of the model is a prediction at each position of what the next phoneme should be. For example, when a well trained model is sees /F IH1/ it should know that /SH/ is likely, or at least not unlikely.

First I'll create a `Vocabulary` instance by passing in all the pronunciations. The `vocab` is used to convert phonemes into integer indices that the neural network handle. It does a few other things too, which you can see below. The `Vocabulary` class's code is in `sonorous/languagemodel.py`.

In [5]:
print(''.join(pronunciations.loc['fish'].pronunciation))

ˈfɪʃ


In [6]:
vocab = Vocabulary.from_texts(pronunciations.pronunciation.values)

print(f"There are {len(vocab)} distinct phonemes.")
print()
print("Looking up the int index for /ʃ/:", vocab['ʃ'])
print()
print("Checking whether /ʃ/ is in the vocabulary:", 'ʃ' in vocab)
print()
print("Looking up the phoneme for a specific int index:", vocab.token_from_idx(vocab['ʃ']))
print()
print("Encoding /ˈfɪʃ/:", vocab.encode_text(tuple("ˈfɪʃ")))

There are 47 distinct phonemes.

Looking up the int index for /ʃ/: 31

Checking whether /ʃ/ is in the vocabulary: True

Looking up the phoneme for a specific int index: ʃ

Encoding /ˈfɪʃ/: [ 1  5 13 22 31  2]


Now I'll define the model. Note that I'm not actually fitting the model to any data so the output will be random. The hyperparameters aren't optimal, but again that doesn't matter here since I just want to show the flow of data through the network.

The `ModelParams` class (from `sonorous/languagemodel.py` encapsulates hyperparameters and options for the model).

In [7]:
model_params = ModelParams(
    rnn_type='rnn', embedding_dimension=10, hidden_dimension=3, num_layers=1,
    max_epochs=3, early_stopping_rounds=3
)

language_model = LanguageModel(vocab, model_params, 'cpu')

Now I'll walk through what happens when we pass the word "fish" /F IH1 SH/ through the model:

In [8]:
fish_pronunciation = tuple("ˈfɪʃ")

The `Vocabulary.encode_text` function we saw earlier does a few things. First, it adds dummy `<START>` and `<END>` tokens to the pronunciation indicating its start and end. This allows the model to learn transition probabilities from the start of the word to the first phoneme, and from the last phoneme to the end of the word.

It then converts every phoneme to its ingeter index.

In [9]:
fish_input = vocab.encode_text(fish_pronunciation)
print(fish_input)
print()
for idx in fish_input:
    phoneme = vocab.token_from_idx(idx)
    print(f'{idx} => {phoneme}')

[ 1  5 13 22 31  2]

1 => <START>
5 => ˈ
13 => f
22 => ɪ
31 => ʃ
2 => <END>


And now we're ready to pass the input into the model's `forward` function, which takes in inputs and outputs predictions. This model's `forward` function expects a Tensor of dimension `(batch_size, NUMBER OF STEPS)`. A step here refers to a step forward in the sequence, so /<START> F IH1 SH <END>/ has 5 steps.

In [10]:
print("Input's shape:", fish_input.shape)
fish_batch_input = torch.LongTensor(fish_input).unsqueeze(0)
print("Batch input's shape:", fish_batch_input.shape)

Input's shape: (6,)
Batch input's shape: torch.Size([1, 6])


The first thing the `forward` function does is embed each phoneme using an [nn.Embedding](https://pytorch.org/docs/stable/nn.html#embedding). Each phoneme has a dedicated embedding vector of length `embedding_dimension`, so the shape of `embedded` is `(batch size, number of steps, embedding_dimension)`.

In [11]:
embedded = language_model._encoder(fish_batch_input)
print(embedded.shape)
print()
embedded

torch.Size([1, 6, 10])



tensor([[[-0.2294, -1.4084,  0.3226, -0.8825, -0.2271,  0.5436, -1.7248,
           1.6166,  0.1770, -0.4132],
         [-0.9655, -0.7882,  0.8303,  0.9622,  0.1076, -1.1665,  0.7774,
           0.3309, -0.0635,  0.7992],
         [ 1.2458,  0.6159, -0.3002,  1.5968,  0.0407,  0.4516,  0.2174,
          -0.9111, -0.1345, -0.7942],
         [ 0.5385, -0.2342,  1.7995,  0.2761, -0.3815, -1.0312, -0.6446,
           1.1468, -0.6206,  1.5344],
         [ 0.1743,  0.5209, -0.2422, -0.4009, -0.3057,  1.1074,  0.9014,
           0.2455,  0.6959,  0.6286],
         [ 0.9723, -1.4500, -0.6116, -0.0278,  0.7446, -1.2008, -1.4380,
          -0.3148,  1.4878,  2.5229]]], grad_fn=<EmbeddingBackward>)

I'll pass `embedded` to the [RNN layer](https://pytorch.org/docs/stable/nn.html#recurrent-layers), resulting in `rnn_output` and `hidden_state`. I won't go into detail on how RNNs work since there are many detailed posts on the web you can read, but the basic idea is a cell is applied sequentially to every token (i.e. step) in the input. At each step an output and a hidden state are produced. The hidden state can be passed on to the next step, and the output can be used to make a prediction.

The `rnn` layer below operates on the full sequence, so the results are for the entire sequence.

In [12]:
rnn_output, hidden_state = language_model._rnn(embedded)
print(rnn_output.shape)
print()
rnn_output

torch.Size([1, 6, 3])



tensor([[[ 0.7130, -0.9789, -0.9616],
         [-0.1030,  0.3504, -0.8634],
         [-0.8661,  0.2290,  0.0268],
         [ 0.6878, -0.9161, -0.5547],
         [-0.3335,  0.4152, -0.3574],
         [ 0.4005, -0.9908,  0.9327]]], grad_fn=<TransposeBackward1>)

For our application we can ignore the `hidden_state`-- the `rnn_output` is the interesting part. The first dimension is for the batch, and we only have a single input in our batch. The second dimension is for each of the input phonemes. The third dimension corresponds to `hidden_dimension`: you can think of this as the state of the RNN at each step.

As I said at the beginning of this section, I want the output of the RNN at each position to be predictions for the *next* position. So I'll apply a [linear layer](https://pytorch.org/docs/stable/nn.html#linear) to the `rnn_output`, resulting in a vector the size of the vocabularly at each position. The [softmax](https://pytorch.org/docs/stable/nn.functional.html#softmax) function normalizes the outputs into probability distributions for each prediction.

In [13]:
outputs = language_model._decoder(rnn_output)
probabilities = F.softmax(outputs, dim=-1).squeeze()
probabilities.shape

torch.Size([6, 47])

The shape of `probabilities` is (5, 42) because each of the five tokens in /ˈfɪʃ/ gets a a probability distribution over each of the 42 phonemes in the vocabulary.

The first phoneme in the input is the `<START>` token; let's see what the model thinks should come next.

In [14]:
probabilities_for_first_phoneme = probabilities[0]
most_likely_first_phoneme_idx = probabilities_for_first_phoneme.argmax().item()
most_likely_first_phoneme = vocab.token_from_idx(most_likely_first_phoneme_idx)
most_likely_first_phoneme

'n'

So the model predicts /ŋ/ to be the first phoneme in the word. Since the model isn't fit yet this is just a random guess. In order to get the model to make good predictions I'll need to first train a good model.

## Finding the Best Model ##
In this section I'll train a number of models on the train set and select the one that has the lowest error on the dev set. I'll split the DataFrame of pronunciations into three DataFrames, with 79% for training, 20% for dev/validation, and 1% for testing of the final model.

In [15]:
train_pronunciations, dev_pronunciations, test_pronunciations = split_data(pronunciations, dev_proportion=.2, test_proportion=.01)
len(train_pronunciations), len(dev_pronunciations), len(test_pronunciations)

(104251, 26393, 1320)

I split the corpus up into train/dev/test so that I could use the standard approach of training models with different hyperparameters and selecting the model that performs best against the dev set. Unfortunately that doesn't work well here because 

Note that I ran a larger parameter search before and saw that GRUs were consistently outperforming LSTMs and vanilla RNNs. There are 12 (4 * 3) models to train, and for each one I'm measuring train and dev error at every epoch. So if each model trains for the maximum of 2,000 epochs I would end up with 12 * 2,000 = 24,000 models to choose from. There's a good chance I'm overfitting the dev set with such a large search, but I'll inspect the learning curves to try to avoid selecting an iteration that randomly did well.

While each model trains for a maximum of 2000 epochs, it stops early if the dev error does not decrease for three epochs in a row. Since I'm going to be selecting the model with the lowest dev error there's no reason to keep training a model once it's started overfitting. Alternatively I could train all models to convergence and then add regularization to reduce the complexity and identify the sweet spot, but that's far more time consuming because it requires training more models and each of them for longer.

In [21]:
%%time

model_parameters = ModelParams(
    rnn_type='gru', num_layers=3, embedding_dimension=20, hidden_dimension=20,
    max_epochs=1000, early_stopping_rounds=3
)

model = LanguageModel(vocab, model_parameters, device_name='cuda')

train_losses, dev_losses = model.fit(
    train_pronunciations.pronunciation.values.tolist(),
    dev_pronunciations.pronunciation.values.tolist(),
    print_every=1,
)

Epoch 1: train loss: 1.1218	dev loss: 1.5515                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 aɪ eɪ z aɪ p v
	 oʊ ɛ
	 aʊ k ŋ θ b ɪ
	 ə m ɔ
	 θ eɪ d
Epoch 2: train loss: 0.9269	dev loss: 1.3255                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 ʒ ʒ ɝ ˈ oʊ ʒ
	 ʃ tʃ
	 
	 ɝ t iː p b aʊ
	 
Epoch 3: train loss: 0.8333	dev loss: 1.2017                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 b ˈ ɝ ʌ t r j m g l t
	 ɔ ˈ oʊ d æ d
	 aʊ tʃ ˈ ɪ z s s l
	 l ɔ s
	 ˈ ɝː s ɝ l aɪ uː
Epoch 4: train loss: 0.7732	dev loss: 1.1180                                                                                               
	Generated: in train: 0%, assess: 1%, novel: 99%
	 ˈ k ʌ k n p ɪ
	 ˈ l
	 ˈ ð aʊ n
	 d ˈ ˈ ɝː z v
	 ˈ v ˌ

Epoch 31: train loss: 0.5580	dev loss: 0.8138                                                                                               
	Generated: in train: 2%, assess: 1%, novel: 97%
	 r ɪ ˈ r iː l
	 ˈ s d ɛ v
	 ˈ s k aɪ ˌ l æ t g ə l i
	 ˈ j ɪ f ə
	 ˌ uː ˌ k oʊ n s ə ə ˈ d iː n ə z
Epoch 32: train loss: 0.5566	dev loss: 0.8118                                                                                               
	Generated: in train: 8%, assess: 2%, novel: 90%
	 ˈ p eɪ k i
	 ˈ b ɛ t ˌ w iː n
	 ˈ h ɪ dʒ ɛ n
	 ˈ ɔ ˌ l æ f s
	 ɛ m ˈ t r aɪ ˌ t ɛ n d
Epoch 33: train loss: 0.5552	dev loss: 0.8098                                                                                               
	Generated: in train: 5%, assess: 3%, novel: 92%
	 ˈ s p ɛ p ɝ ə z
	 ˈ w æ s t ə l p s
	 ˈ w æ l s iː z
	 ˈ k l æ n t ɪ ŋ
	 ˈ s t j aɪ dʒ oʊ
Epoch 34: train loss: 0.5540	dev loss: 0.8083                                                                                               
	Generat

Epoch 60: train loss: 0.5360	dev loss: 0.7827                                                                                               
	Generated: in train: 14%, assess: 1%, novel: 85%
	 p æ ˈ k j uː
	 ˈ k eɪ d
	 ˈ s w oʊ l t ɝ
	 ˌ æ d ə ˈ l j ɪ ð ə
	 ə l ˈ t ɔ r m ˌ n ɛ t
Epoch 61: train loss: 0.5358	dev loss: 0.7824                                                                                               
	Generated: in train: 13%, assess: 1%, novel: 86%
	 ˈ d ɛ l ɝ d
	 m ə ˈ m ɝː d ə l ɪ z
	 ɛ ˈ r ʊ v ɪ v
	 m aɪ ˈ ɑː d oʊ
	 ˈ p l æ s n ə
Epoch 62: train loss: 0.5352	dev loss: 0.7815                                                                                               
	Generated: in train: 7%, assess: 1%, novel: 92%
	 ˈ l ʊ p
	 t r uː ə ˈ k oʊ d w eɪ t ɪ k
	 ˈ s t l ɑː k l ɔ tʃ
	 ˈ s æ n ɪ ŋ
	 s p uː ˈ g oʊ ɪ s t
Epoch 63: train loss: 0.5347	dev loss: 0.7809                                                                                               
	Generated: i

Epoch 89: train loss: 0.5275	dev loss: 0.7710                                                                                               
	Generated: in train: 13%, assess: 1%, novel: 86%
	 ˈ k oʊ s ɪ ə n ɑː n
	 m ɪ g ə m ˈ s iː l
	 ˈ d ʌ s t j ɝ
	 k ə ˈ l ɔ s
	 ˈ g r iː d z
Epoch 90: train loss: 0.5273	dev loss: 0.7709                                                                                               
	Generated: in train: 15%, assess: 1%, novel: 84%
	 ˈ d aɪ k r ə n
	 ˈ h iː d ə n
	 ˈ d æ t ɪ s t
	 ˈ n ɛ l z
	 ˈ s t r eɪ b ɪ s n ə n t
Epoch 91: train loss: 0.5270	dev loss: 0.7704                                                                                               
	Generated: in train: 11%, assess: 1%, novel: 88%
	 ˈ l ʌ ŋ ə s t s
	 ˈ ɝː t oʊ
	 m uː ˈ ɑː l uː
	 ɛ k r ɑː ˈ f ɪ v i
	 ˈ b r ɪ k ɝ ˌ ɑː l d ɝ
Epoch 92: train loss: 0.5269	dev loss: 0.7702                                                                                               
	Generated: in tra

Epoch 118: train loss: 0.5234	dev loss: 0.7656                                                                                               
	Generated: in train: 7%, assess: 3%, novel: 90%
	 ˈ s iː d ɝ
	 ˈ t æ b s k ə
	 ˈ j ʊ r ɑː l
	 ɑː k r ɑː ˈ t ɔ r ə n
	 h ɑː r ˈ m ɪ l ə
Epoch 119: train loss: 0.5230	dev loss: 0.7650                                                                                               
	Generated: in train: 12%, assess: 2%, novel: 86%
	 ˈ k eɪ d ə l
	 ˌ d ɪ s ɑː ˈ r ɪ k iː iː iː z
	 ˈ t ɪ l oʊ
	 ə ˈ tʃ ɪ t ɪ v
	 ˈ tʃ ɝː b ɝ
Epoch 120: train loss: 0.5229	dev loss: 0.7648                                                                                               
	Generated: in train: 12%, assess: 3%, novel: 85%
	 ˈ p l iː ə
	 ˈ ɛ l k ˌ ʃ iː l d
	 ˈ b r aɪ k s
	 oʊ ˈ l eɪ
	 d ɪ ˈ r uː n i
Epoch 121: train loss: 0.5228	dev loss: 0.7647                                                                                               
	Generated: in train: 10%, 

...

Epoch 205: train loss: 0.5166	dev loss: 0.7566                                                                                               
	Generated: in train: 13%, assess: 5%, novel: 82%
	 ˈ b ɝː dʒ ə n
	 ˈ p r ɑː s ə n d z
	 ˈ dʒ æ k ˌ s eɪ z
	 ˈ t r ʌ m ɪ s t r ə n t
	 ˈ b ɛ l ə n d z
Epoch 206: train loss: 0.5166	dev loss: 0.7565                                                                                               
	Generated: in train: 14%, assess: 1%, novel: 85%
	 ˈ k ɑː l i
	 ˈ k ɑː g ˌ k r eɪ t
	 f ɝ ˈ iː d
	 m ə k ˈ g r æ w iː ə l
	 ˈ p ɔ r g i
Epoch 207: train loss: 0.5165	dev loss: 0.7563                                                                                               
	Generated: in train: 14%, assess: 3%, novel: 83%
	 g r ɪ ˈ l ɪ ʃ ə n tʃ
	 ˈ æ s t ə h eɪ n z
	 ˌ s ɪ r ə ˈ p oʊ ʃ ə n z
	 ˈ g ʌ b l ɝ s iː z
	 ɝ ˈ z ɪ s t ə n
Epoch 208: train loss: 0.5169	dev loss: 0.7569                                                                                

Epoch 234: train loss: 0.5157	dev loss: 0.7555                                                                                               
	Generated: in train: 14%, assess: 1%, novel: 85%
	 k ə ˌ b l oʊ s d ə ˈ k eɪ ʃ ə n
	 p ɑː ˈ m ɪ s ə ˌ l ɪ s t s
	 ˈ d æ b
	 ˈ b l ɔ r ɝ
	 ˈ r oʊ s ə
Epoch 235: train loss: 0.5156	dev loss: 0.7553                                                                                               
	Generated: in train: 23%, assess: 2%, novel: 75%
	 ə ˈ z ɔ r d
	 ˈ s ʌ m b oʊ
	 ˈ t æ n t z ɝ
	 ˈ k æ n d ˌ s k ɔ r z
	 ˌ t ɪ n ə ˈ m ɪ t ə n s
Epoch 236: train loss: 0.5154	dev loss: 0.7550                                                                                               
	Generated: in train: 13%, assess: 1%, novel: 86%
	 ˈ h æ m
	 ˈ s t ɛ b d ə n z
	 ˈ m ɪ m ə l
	 b r oʊ ˈ b ʌ k t ɪ ŋ
	 ˌ dʒ ɑː ˈ n ɑː r ə
Epoch 237: train loss: 0.5153	dev loss: 0.7550                                                                                               

Epoch 263: train loss: 0.5147	dev loss: 0.7543                                                                                               
	Generated: in train: 17%, assess: 4%, novel: 79%
	 ˈ m aɪ g w ɝ k s
	 k ə ˈ t aɪ n ɪ k
	 ˈ n ɛ g ɝ
	 ˈ p ɛ g ɝ ˌ f aɪ b j ə z
	 ˈ d ɔ r d ə
Epoch 264: train loss: 0.5146	dev loss: 0.7541                                                                                               
	Generated: in train: 16%, assess: 1%, novel: 83%
	 ˈ ʌ d n ɝ
	 ˈ ʃ l æ p
	 ˈ r uː tʃ ɪ l
	 ˌ ɪ n t ɝ ˈ s ɛ l d
	 ˈ d uː l ɪ n z z
Epoch 265: train loss: 0.5145	dev loss: 0.7538                                                                                               
	Generated: in train: 9%, assess: 3%, novel: 88%
	 ˌ m ɑː n d ə ˈ m æ dʒ ə ˌ t iː d
	 ˈ d ɛ h ɝ s ə n
	 ˈ tʃ uː n ɝ
	 v ɑː ˈ r ɑː r ɪ k i
	 ˈ tʃ ɪ z ə m
Epoch 266: train loss: 0.5146	dev loss: 0.7540                                                                                               
	Genera

Epoch 292: train loss: 0.5139	dev loss: 0.7533                                                                                               
	Generated: in train: 16%, assess: 2%, novel: 82%
	 ˈ ɔ b ɝ oʊ z
	 æ l ˈ m eɪ
	 p l ə ˈ p ɪ ʃ ə n z
	 ˈ b l oʊ d m ə n
	 ˈ p r ɝː n ˌ f aɪ n
Epoch 293: train loss: 0.5141	dev loss: 0.7533                                                                                               
	Generated: in train: 13%, assess: 1%, novel: 86%
	 ɑː n j iː ˈ s t ɔ r i
	 ˈ m ɪ l θ ɪ n
	 ˌ oʊ b ɝ ˈ eɪ ʃ ɝ
	 m ɪ ˈ s t eɪ n z
	 ˈ h ɛ l
Epoch 294: train loss: 0.5137	dev loss: 0.7530                                                                                               
	Generated: in train: 19%, assess: 2%, novel: 79%
	 ˈ p l uː n j ə n
	 ˈ s uː z ɪ ŋ
	 ˈ f ɪ ŋ k n ə
	 ˈ t ɪ ŋ g ə l
	 ˈ ʃ ɑː l i
Epoch 295: train loss: 0.5138	dev loss: 0.7531                                                                                               
	Generated: in train: 1

Early stopping because of no decrease in 3 epochs.


In [22]:
with open('model_20_20_3.pt', 'wb') as fh:
    model.save(fh)

In [23]:
%%time

model_parameters = ModelParams(
    rnn_type='gru', num_layers=1, embedding_dimension=20, hidden_dimension=20,
    max_epochs=1000, early_stopping_rounds=3
)

model2 = LanguageModel(vocab, model_parameters, device_name='cuda')

train_losses2, dev_losses2 = model2.fit(
    train_pronunciations.pronunciation.values.tolist(),
    dev_pronunciations.pronunciation.values.tolist(),
    print_every=1,
)

Epoch 1: train loss: 1.1494	dev loss: 1.5605                                                                                               
	Generated: in train: 1%, assess: 0%, novel: 99%
	 ʊ oʊ w ʃ
	 ɝ
	 z m k ð
	 dʒ
	 aɪ ð p l oʊ m j h ɔ ɝ g p m aʊ aɪ r i ɔɪ dʒ aɪ eɪ ɪ t tʃ
Epoch 2: train loss: 0.9021	dev loss: 1.2833                                                                                               
	Generated: in train: 2%, assess: 0%, novel: 98%
	 
	 
	 
	 w i s ŋ d
	 ɪ w g b d æ θ b z æ g ð
Epoch 3: train loss: 0.8014	dev loss: 1.1505                                                                                               
	Generated: in train: 2%, assess: 0%, novel: 98%
	 ˈ tʃ ɔɪ
	 v ˈ ɑː w l k n ə ɑː n
	 ʒ dʒ
	 eɪ ˈ ɑː n r
	 ˈ p b h ɑː
Epoch 4: train loss: 0.7502	dev loss: 1.0817                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 ˈ g eɪ r t
	 s oʊ z
	 ˈ g z eɪ ʒ ɛ b


Epoch 31: train loss: 0.5555	dev loss: 0.8095                                                                                               
	Generated: in train: 13%, assess: 2%, novel: 85%
	 ˈ f r iː ˌ d æ n
	 k ə n ˈ h ɪ m oʊ dʒ
	 ˈ s uː n ɪ s
	 ˈ w ʌ k j ə n t
	 ˈ h æ t i
Epoch 32: train loss: 0.5546	dev loss: 0.8083                                                                                               
	Generated: in train: 7%, assess: 1%, novel: 92%
	 ˌ ɑː n f iː k ə ˈ t l eɪ s
	 ˈ s t l oʊ ˌ t ɑː g
	 m ɑː m t r iː ˈ eɪ k w iː t l oʊ
	 ˈ r ɔ ŋ k n ɝ
	 s ˈ s ɪ l ɝ d
Epoch 33: train loss: 0.5538	dev loss: 0.8071                                                                                               
	Generated: in train: 2%, assess: 2%, novel: 96%
	 ˈ r ʊ r ˌ k s ɪ p s t ə n
	 ˈ n ʌ m ɪ s k
	 ˈ p æ n ɝ
	 ˈ k oʊ n ɝ z
	 ˈ l ɛ t ə m ə n
Epoch 34: train loss: 0.5530	dev loss: 0.8060                                                                                           

Epoch 60: train loss: 0.5399	dev loss: 0.7872                                                                                               
	Generated: in train: 14%, assess: 2%, novel: 84%
	 d iː ˈ z iː d b ɪ k s ɝ i
	 ˈ m ɑː d r ɪ t
	 ˈ b ɛ z k r iː l
	 ɛ r ˈ v æ t ə n
	 ˈ g r eɪ s ə n
Epoch 61: train loss: 0.5396	dev loss: 0.7868                                                                                               
	Generated: in train: 11%, assess: 3%, novel: 86%
	 ˈ p r ʌ m eɪ z
	 ˈ s oʊ l ɪ v
	 p ɪ ˈ l æ t z
	 ˈ s t æ k ɝ
	 ˈ s uː p t ɝ
Epoch 62: train loss: 0.5393	dev loss: 0.7864                                                                                               
	Generated: in train: 13%, assess: 0%, novel: 87%
	 ˈ p r ʌ m ˌ w ɛ z
	 ˈ k ɛ l m ə n
	 ˈ f ɔ r ˌ k l eɪ t
	 h ɑː ˈ k ɛ r s iː oʊ z
	 ˈ h æ k
Epoch 63: train loss: 0.5390	dev loss: 0.7859                                                                                               
	Generated: in tra

Epoch 89: train loss: 0.5346	dev loss: 0.7797                                                                                               
	Generated: in train: 15%, assess: 2%, novel: 83%
	 s ɑː n ˈ d ɑː r t ə n i
	 ˈ w ɪ n s
	 ˌ ɔ θ ɪ ˈ s t ɑː k
	 ˈ l ɑː dʒ ə z
	 ˈ ɛ n d ɝ d
Epoch 90: train loss: 0.5346	dev loss: 0.7796                                                                                               
	Generated: in train: 19%, assess: 3%, novel: 78%
	 ˈ h ɛ r i
	 ˈ k l æ m p
	 ˈ n ɪ k t
	 ˈ r iː v oʊ ˌ f ɛ t
	 ˌ iː ɑː ˈ l ɑː r iː oʊ
Epoch 91: train loss: 0.5345	dev loss: 0.7795                                                                                               
	Generated: in train: 10%, assess: 2%, novel: 88%
	 ˈ g æ l k ə
	 ˈ d uː ɪ ŋ
	 ˈ v ɑː r iː z
	 ˌ m ɪ n ˈ s eɪ ʃ ə l d
	 ˈ dʒ ɑː l
Epoch 92: train loss: 0.5343	dev loss: 0.7793                                                                                               
	Generated: in train: 8%, assess

Epoch 118: train loss: 0.5321	dev loss: 0.7761                                                                                               
	Generated: in train: 14%, assess: 2%, novel: 84%
	 ˌ ɪ b ɝ ˈ ɛ r i
	 ˈ s t ʌ p s
	 ˈ s ɛ l t ɝ b ɝ
	 ˌ p ɛ d ɛ ˈ n oʊ n d
	 d oʊ ˈ m eɪ p ɝ l i
Epoch 119: train loss: 0.5320	dev loss: 0.7760                                                                                               
	Generated: in train: 10%, assess: 3%, novel: 87%
	 ˈ b oʊ d b ɝ ɝ
	 r ɔ h ˈ r ɔɪ z
	 ˈ k ɛ r m ə l i
	 ˈ j ɑː n t t
	 ˈ s ɛ t s
Epoch 120: train loss: 0.5319	dev loss: 0.7758                                                                                               
	Generated: in train: 9%, assess: 1%, novel: 90%
	 ˌ ɛ ˈ z ɛ l i
	 b ɪ ˈ r ɛ s ə m ˌ aɪ t
	 ˈ s ɪ v l ɝ k s
	 ˌ t ə n ə t ˌ m ɑː n s ɪ ˈ r ɛ l t ɪ k ɝ
	 ˈ b ɝː v s
Epoch 121: train loss: 0.5318	dev loss: 0.7757                                                                                          

Epoch 147: train loss: 0.5302	dev loss: 0.7737                                                                                               
	Generated: in train: 11%, assess: 1%, novel: 88%
	 v ɛ n ˈ ɔ r k
	 ˈ j iː n ɝ s ə n
	 ˈ p ʊ r ˌ k eɪ m
	 ˈ æ k s iː ˌ ʌ f d
	 k ə ˈ b ɪ l f ə l
Epoch 148: train loss: 0.5301	dev loss: 0.7734                                                                                               
	Generated: in train: 8%, assess: 3%, novel: 89%
	 ˈ f ɛ ˌ s t iː f
	 ˈ w ɑː p s i
	 r ɑː ˈ t ɛ n tʃ i
	 d ɪ ˈ l ɛ z ˌ n ɛ g
	 ˈ b oʊ l ɝ
Epoch 149: train loss: 0.5301	dev loss: 0.7736                                                                                               
	Generated: in train: 14%, assess: 0%, novel: 86%
	 ˈ ʌ n t i
	 ˈ b oʊ l h uː t s
	 ˈ t uː
	 ˈ d aɪ m ɪ n
	 ˈ m æ l i
Epoch 150: train loss: 0.5301	dev loss: 0.7734                                                                                               
	Generated: in train: 7%, asses

Epoch 176: train loss: 0.5290	dev loss: 0.7721                                                                                               
	Generated: in train: 8%, assess: 1%, novel: 91%
	 ˈ ʃ oʊ v iː eɪ
	 ˈ h ɛ n s ə b l ɝ
	 ˈ h oʊ m ɝ
	 ˈ g ɑː r s ɪ d z
	 ˈ d ɪ n ˌ f r aʊ m ɝ
Epoch 177: train loss: 0.5290	dev loss: 0.7721                                                                                               
	Generated: in train: 12%, assess: 2%, novel: 86%
	 ˈ w ɪ l ɪ k i
	 ˈ d ɑː k t
	 ˌ ɪ f ˈ ɛ l iː
	 g uː ˈ oʊ t iː
	 ˌ m ɛ f ə ˈ p r eɪ ʃ ə n
Epoch 178: train loss: 0.5289	dev loss: 0.7720                                                                                               
	Generated: in train: 10%, assess: 1%, novel: 89%
	 ə ˈ b ɛ k l ɪ ʃ ə n
	 ˈ ɔ l ə k s
	 ˈ r ɛ p t
	 ˈ m aɪ t s
	 ˈ eɪ n z
Epoch 179: train loss: 0.5289	dev loss: 0.7719                                                                                               
	Generated: in train: 9%, ass

Early stopping because of no decrease in 3 epochs.


In [25]:
with open('model_20_20_1.pt', 'wb') as fh:
    model2.save(fh)

In [24]:
%%time

model_parameters = ModelParams(
    rnn_type='gru', num_layers=1, embedding_dimension=20, hidden_dimension=10,
    max_epochs=1000, early_stopping_rounds=3
)

small_model = LanguageModel(vocab, model_parameters, device_name='cuda')

_ = small_model.fit(
    train_pronunciations.pronunciation.values.tolist(),
    dev_pronunciations.pronunciation.values.tolist(),
    print_every=1,
)

Epoch 1: train loss: 2.1104	dev loss: 2.3951                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 i ˌ uː i j i ɝ ɔɪ θ b
	 ˌ
	 m ɔɪ ʌ iː θ aɪ ʌ j aɪ ɔ ʊ ɛ ˈ iː ɑː ð æ g s ɝː f
	 f æ ɔ aʊ ð aɪ
	 oʊ z ʊ l aɪ ɝː ɑː ɝː tʃ l i m
Epoch 2: train loss: 1.2157	dev loss: 1.5925                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 w b aʊ ŋ
	 æ ə ʌ ˈ j uː r t uː eɪ uː i dʒ
	 iː w
	 θ
	 ɝ ˈ ŋ z i
Epoch 3: train loss: 0.9858	dev loss: 1.3592                                                                                               
	Generated: in train: 1%, assess: 0%, novel: 99%
	 p ˈ m ɝ l uː ʌ s l
	 
	 
	 ˈ
	 ɝ ˈ ɛ r w ʌ θ h θ ʃ
Epoch 4: train loss: 0.8879	dev loss: 1.2491                                                                                               
	Generated: in train: 1%, assess: 0%, novel:

Epoch 31: train loss: 0.5902	dev loss: 0.8585                                                                                               
	Generated: in train: 14%, assess: 1%, novel: 85%
	 ˌ ɑː ˈ k ɛ m ɪ s
	 ˈ dʒ ʌ k ə k s i
	 k p r ɝ ˈ ð ʌ l t l ə w
	 ˈ f l j uː s k
	 t r oʊ ˈ s j s aʊ l θ
Epoch 32: train loss: 0.5887	dev loss: 0.8564                                                                                               
	Generated: in train: 8%, assess: 0%, novel: 92%
	 b ə ˈ l æ n h ə ˌ h æ n s
	 ˈ t ɑː n i
	 ˈ l b ɔ r l ə dʒ
	 ˈ b ɔ l ɝ z
	 ˈ k s oʊ s t ə n ŋ
Epoch 33: train loss: 0.5874	dev loss: 0.8545                                                                                               
	Generated: in train: 4%, assess: 1%, novel: 95%
	 ˈ m iː r ʊ
	 s ə ˈ l m ɪ f ə n z z
	 ˈ tʃ ɪ r ɪ s
	 ˈ r iː m
	 ˈ p ɪ n d ɛ l ə n
Epoch 34: train loss: 0.5861	dev loss: 0.8527                                                                                               
	Gene

Epoch 60: train loss: 0.5676	dev loss: 0.8266                                                                                               
	Generated: in train: 10%, assess: 2%, novel: 88%
	 ɪ ˈ d ɪ p l iː ə
	 ˌ s ɛ k s ɪ ˈ b ɛ k n i
	 ˈ m ɑː r j ɝː ɝ
	 ˈ h r ɪ l k s s
	 ə m ˈ f ɛ r i
Epoch 61: train loss: 0.5671	dev loss: 0.8260                                                                                               
	Generated: in train: 9%, assess: 3%, novel: 88%
	 ˈ w ɔ r n ɝ
	 ˈ k ɑː n ʃ ɪ n
	 m iː t æ ˌ r iː m ɪ ˈ n iː k oʊ
	 ˈ r ɔ l f ˌ s m ɪ t
	 ˈ m ɪ k θ ə ˌ d ɪ s t t
Epoch 62: train loss: 0.5667	dev loss: 0.8255                                                                                               
	Generated: in train: 14%, assess: 0%, novel: 86%
	 ˈ r æ m r ə n
	 ˈ d ɑː ʃ ə k
	 ˈ eɪ d ə n z
	 ˌ ɛ r iː d ɝ ˈ aɪ ə n i
	 s ɛ n ˈ k ɔ n ə l
Epoch 63: train loss: 0.5663	dev loss: 0.8249                                                                                 

Epoch 89: train loss: 0.5595	dev loss: 0.8152                                                                                               
	Generated: in train: 15%, assess: 1%, novel: 84%
	 ˈ d ɑː m ə n
	 r ɔ ˈ ɛ k
	 ˈ k ɑː l d t i
	 ˌ r uː k ə ˌ n ɛ r eɪ t ˈ w ʌ tʃ ə n
	 ˈ s ɪ k s
Epoch 90: train loss: 0.5593	dev loss: 0.8149                                                                                               
	Generated: in train: 9%, assess: 1%, novel: 90%
	 ˈ h ɪ s t ɪ
	 ˈ k ɪ l ˌ b aɪ z
	 ˌ ɛ r ˌ k r ɪ d ˈ k iː n ɪ ˌ b l eɪ
	 ˈ l ɔ r ɪ ŋ
	 ˈ k r æ v ɪ ŋ
Epoch 91: train loss: 0.5591	dev loss: 0.8146                                                                                               
	Generated: in train: 12%, assess: 0%, novel: 88%
	 ˌ f aɪ ˈ æ t r ə t ə l d
	 ˈ ʃ ʌ k ə r z
	 ˌ s ɔ p ɪ k ˈ l ɪ p ɪ ŋ ˌ ɛ m z
	 ˌ f ɛ s ɑː ˈ w æ m ɪ ŋ
	 k uː ˈ n ɑː l ə d
Epoch 92: train loss: 0.5589	dev loss: 0.8143                                                                 

Epoch 118: train loss: 0.5547	dev loss: 0.8086                                                                                               
	Generated: in train: 9%, assess: 1%, novel: 90%
	 r ə ˈ s ʌ b ə w ɪ l
	 ˈ b r eɪ n m ɑː ˌ l ɪ t s
	 ˈ b r æ p ə s t
	 ɝ ˈ aʊ n b ɝ d
	 ˈ dʒ ɝː p
Epoch 119: train loss: 0.5545	dev loss: 0.8083                                                                                               
	Generated: in train: 8%, assess: 1%, novel: 91%
	 ˈ l oʊ z ˌ g ɑː r z
	 ˈ h ɑː v i
	 ˈ θ ʊ r d t
	 k ə ˈ k ɪ r ə s
	 ˈ dʒ ʌ b
Epoch 120: train loss: 0.5545	dev loss: 0.8082                                                                                               
	Generated: in train: 12%, assess: 1%, novel: 87%
	 ˈ k ɔ r k i
	 ˈ k æ t ɝ ˌ ɑː r z
	 ˈ r eɪ ˌ m eɪ p t
	 ɛ ˈ ʃ ɛ n ɪ tʃ i
	 ˈ s t l oʊ k ˌ θ aɪ z
Epoch 121: train loss: 0.5544	dev loss: 0.8081                                                                                               
	Generated:

...

Epoch 349: train loss: 0.5465	dev loss: 0.7965                                                                                               
	Generated: in train: 10%, assess: 1%, novel: 89%
	 ˈ ɑː n z oʊ
	 ˈ m ɛ s t ɝ ɝ
	 ˈ b ɝː k
	 ˌ s uː ˈ t ɑː r iː ˌ b æ s
	 ˈ v ɪ n d ɝ ə n ˌ s eɪ t iː n
Epoch 350: train loss: 0.5465	dev loss: 0.7965                                                                                               
	Generated: in train: 11%, assess: 1%, novel: 88%
	 ˈ h æ k ə b z
	 r ɪ ˈ f oʊ t ɝ
	 ˈ s ɑː k s ə n
	 ˈ ʃ w ɛ n m ə t s ə
	 d oʊ ˈ l oʊ tʃ ɝ z
Epoch 351: train loss: 0.5466	dev loss: 0.7966                                                                                               
	Generated: in train: 12%, assess: 0%, novel: 88%
	 k ə ˈ n ɛ z b
	 ˈ r eɪ ˌ n æ z oʊ
	 g ɝ ˈ h r oʊ l ɪ d
	 ˈ d ɛ t s i
	 ˌ k ɛ r ɪ g ˈ l ɛ n p s
Epoch 352: train loss: 0.5465	dev loss: 0.7965                                                                                      

Epoch 378: train loss: 0.5462	dev loss: 0.7959                                                                                               
	Generated: in train: 3%, assess: 2%, novel: 95%
	 k ɝ ˈ ɔ r k l i
	 ˈ t ʌ p ɪ ŋ
	 d iː ˈ g ɑː z oʊ
	 d ɝ m ˈ l oʊ l l
	 ˈ d ɑː n i
Epoch 379: train loss: 0.5461	dev loss: 0.7959                                                                                               
	Generated: in train: 6%, assess: 5%, novel: 89%
	 m ə ˈ n æ n ɪ l
	 ˈ ɪ l s ə n
	 m ə k ˈ s p oʊ b ə t s
	 v ɝː k k ɪ s t ˈ n iː k ˌ t aɪ d iː z
	 b ə ˈ r aɪ b ˌ l æ v z
Epoch 380: train loss: 0.5462	dev loss: 0.7960                                                                                               
	Generated: in train: 7%, assess: 1%, novel: 92%
	 ˌ m ɛ r ə ˈ m iː z iː n ɪ ŋ
	 ˈ l æ n oʊ t oʊ
	 ˈ m ɛ r ɪ k
	 ˈ d aʊ m ə n d
	 ˈ b ʊ l d ə n
Epoch 381: train loss: 0.5462	dev loss: 0.7961                                                                                 

Epoch 407: train loss: 0.5459	dev loss: 0.7955                                                                                               
	Generated: in train: 13%, assess: 3%, novel: 84%
	 ˈ ɪ m ɝ v ɔ g
	 ˈ k ɑː l ɝ
	 ˈ p r oʊ d d
	 ˌ k ɛ m ʊ ˈ b ɔ r t ɪ v ɪ ŋ
	 ˌ t ɛ r iː ˈ m ɑː l z
Epoch 408: train loss: 0.5459	dev loss: 0.7956                                                                                               
	Generated: in train: 10%, assess: 1%, novel: 89%
	 p m ɑː ˈ n ə n dʒ r ə n
	 ˈ h ɪ d ɝ
	 ɛ ˈ l ɛ m ə n z
	 ˈ s uː t ɪ t
	 ˈ m ɔ r ə d
Epoch 409: train loss: 0.5459	dev loss: 0.7955                                                                                               
	Generated: in train: 17%, assess: 2%, novel: 81%
	 ˈ m æ n b ɪ dʒ ə
	 p ɝ ˈ d iː k
	 ˈ p ɑː l iː ˌ æ n
	 ˈ j uː l ə d
	 ˈ r aɪ n ɪ k
Epoch 410: train loss: 0.5458	dev loss: 0.7955                                                                                               
	Generated: in 

Epoch 436: train loss: 0.5456	dev loss: 0.7952                                                                                               
	Generated: in train: 14%, assess: 1%, novel: 85%
	 ˈ b iː d b ɝ l ɑː k
	 ˈ ʒ aɪ n i
	 ˌ ʃ ɑː n ˈ t r ɑː tʃ s k i
	 ˈ s ɑː l z oʊ l eɪ t
	 ˈ d aɪ z t ɝ ˌ tʃ aɪ
Epoch 437: train loss: 0.5457	dev loss: 0.7953                                                                                               
	Generated: in train: 19%, assess: 0%, novel: 81%
	 ˈ tʃ ɑː d ə n
	 ˈ s t ʊ r d
	 ˈ f ɛ ʃ s
	 ˈ w iː r ə
	 ˈ s t ɑː l ɝ
Epoch 438: train loss: 0.5456	dev loss: 0.7952                                                                                               
	Generated: in train: 9%, assess: 0%, novel: 91%
	 d ɪ ˈ v ɪ ŋ g i
	 ˈ m iː t r ɪ ŋ
	 ˈ h r ɪ t ə d
	 ˈ t r ɪ s ə ˌ w iː z z
	 ˈ k æ m ˌ f l æ z
Epoch 439: train loss: 0.5456	dev loss: 0.7951                                                                                               
	Genera

Early stopping because of no decrease in 3 epochs.


In [26]:
with open('model_20_10_1.pt', 'wb') as fh:
    small_model.save(fh)

In [27]:
%%time

model_parameters = ModelParams(
    rnn_type='gru', num_layers=1, embedding_dimension=10, hidden_dimension=10,
    max_epochs=1000, early_stopping_rounds=3
)

model_10_10_1 = LanguageModel(vocab, model_parameters, device_name='cuda')

_ = model_10_10_1.fit(
    train_pronunciations.pronunciation.values.tolist(),
    dev_pronunciations.pronunciation.values.tolist(),
    print_every=1,
)

with open('model_10_10_1.pt', 'wb') as fh:
    model_10_10_1.save(fh)

Epoch 1: train loss: 2.1243	dev loss: 2.4062                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 i l ə æ ɝː dʒ z
	 ʊ ɔɪ
	 b uː b ˈ iː tʃ ɝ ð ˈ v ɔ p tʃ t v z ɑː v ˌ ʊ ɪ
	 r ɔ l f æ l tʃ ɝ ŋ ɔ p j
	 
Epoch 2: train loss: 1.2583	dev loss: 1.6582                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 b k
	 oʊ oʊ
	 p ɔ ð p aʊ ŋ ɝː p ŋ ɑː ɝː
	 
	 w aʊ
Epoch 3: train loss: 1.0518	dev loss: 1.4565                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 z oʊ t ˌ w k ɝː æ
	 ʊ ɔɪ z
	 tʃ ɪ ɔ p d ð k
	 iː
	 r aɪ ŋ ˈ ɪ n s ɪ ə dʒ
Epoch 4: train loss: 0.9470	dev loss: 1.3357                                                                                               
	Generated: in train: 1%, assess: 0%, novel: 99%
	 
	 j
	 w

Epoch 31: train loss: 0.5940	dev loss: 0.8642                                                                                               
	Generated: in train: 7%, assess: 1%, novel: 92%
	 ˈ aɪ d iː t ɪ b ə z
	 ˈ r ɑː ˌ b r eɪ t
	 ə ˈ p æ n ˌ s æ g ɝ
	 ˈ æ t z iː z
	 k ɔ ˈ d æ t ə
Epoch 32: train loss: 0.5926	dev loss: 0.8623                                                                                               
	Generated: in train: 14%, assess: 0%, novel: 86%
	 ˈ w iː s g ɝ n
	 ˈ m ɑː ɝ z
	 ˈ f ə s z ɪ k b ə n s ɪ s
	 ˈ m ə t
	 ˈ p æ r ə n
Epoch 33: train loss: 0.5912	dev loss: 0.8604                                                                                               
	Generated: in train: 12%, assess: 1%, novel: 87%
	 ˈ h iː g ɝ z ə k g
	 ˈ f r ɑː r
	 ˈ l ʌ s t ɝ z
	 m iː ˈ k ɑː l f ə n
	 ˌ m ɑː ˈ r l ɑː l
Epoch 34: train loss: 0.5899	dev loss: 0.8586                                                                                               
	Generated: in tra

Epoch 60: train loss: 0.5689	dev loss: 0.8290                                                                                               
	Generated: in train: 15%, assess: 1%, novel: 84%
	 ˈ t r ɛ k oʊ
	 ˈ d iː v ɝ z
	 ɑː r ˈ p ɑː n v ə
	 ˈ d æ k
	 ˈ k ɛ n i
Epoch 61: train loss: 0.5685	dev loss: 0.8283                                                                                               
	Generated: in train: 9%, assess: 1%, novel: 90%
	 ˈ n p eɪ z
	 ˌ ɛ n p ə m ˈ eɪ g iː ɝ
	 ˈ k r ʊ v ɝ ˌ p ɛ l ɔɪ
	 ˈ n æ s t ɝ d
	 ˌ m iː ˈ h ɑː s t dʒ ɝ
Epoch 62: train loss: 0.5681	dev loss: 0.8277                                                                                               
	Generated: in train: 16%, assess: 2%, novel: 82%
	 m ə ˈ n ɪ r ɪ z
	 ˈ l æ n d
	 ˌ ɑː n ˈ k æ s t s
	 ˈ g ɪ t r aɪ ɝ
	 ˈ k oʊ l ə z
Epoch 63: train loss: 0.5677	dev loss: 0.8273                                                                                               
	Generated: in train: 13%, 

Epoch 89: train loss: 0.5611	dev loss: 0.8179                                                                                               
	Generated: in train: 10%, assess: 2%, novel: 88%
	 ˈ l oʊ d
	 ˈ z iː n ɪ d
	 ˌ ˌ p ɑː r ə ˈ n ɑː k ə n
	 ˈ ɑː n ʃ ɪ t b ɝ
	 ˈ b r ɔ ˌ b j ɑː g
Epoch 90: train loss: 0.5609	dev loss: 0.8177                                                                                               
	Generated: in train: 14%, assess: 1%, novel: 85%
	 ˈ k r iː z
	 ˈ f r aʊ dʒ z
	 ˈ k ʊ b r ə m
	 ə n ˈ d æ n z ɝ
	 ˈ b ɛ k ə t
Epoch 91: train loss: 0.5608	dev loss: 0.8175                                                                                               
	Generated: in train: 14%, assess: 0%, novel: 86%
	 ˈ ɛ l ə
	 k ə ˈ tʃ uː t ə n
	 ˈ r æ k ˌ s m oʊ d
	 ˌ oʊ ˈ m æ n
	 ˈ m ɛ n t ɪ m
Epoch 92: train loss: 0.5607	dev loss: 0.8174                                                                                               
	Generated: in train: 13%, assess

Epoch 118: train loss: 0.5574	dev loss: 0.8126                                                                                               
	Generated: in train: 10%, assess: 1%, novel: 89%
	 ˈ s oʊ k
	 b r ɪ g ˈ t æ r ɪ ŋ
	 ˈ g r ɑː n ə
	 m ɝ ɛ f ˈ g r iː n ˌ ɛ r
	 ˈ m ɪ g ɝ
Epoch 119: train loss: 0.5574	dev loss: 0.8125                                                                                               
	Generated: in train: 11%, assess: 1%, novel: 88%
	 ˈ s p ɛ m p l ɪ d oʊ z
	 m r ə ˈ s ɛ n
	 ˈ r æ n b ə l b ɝ ˈ ɛ l d
	 ˈ b ɛ m ə t j ɪ n
	 ˈ tʃ æ b ɝ z
Epoch 120: train loss: 0.5572	dev loss: 0.8123                                                                                               
	Generated: in train: 16%, assess: 3%, novel: 81%
	 ˈ s k oʊ t r iː d
	 ˈ r ɪ s g ɝ ˌ t l uː d
	 ˈ g ɑː n ə n s t
	 ˈ r oʊ n
	 ˈ k ɑː r ɔ r ə f
Epoch 121: train loss: 0.5571	dev loss: 0.8121                                                                                             

...

Epoch 292: train loss: 0.5495	dev loss: 0.8012                                                                                               
	Generated: in train: 6%, assess: 2%, novel: 92%
	 ˈ k r ʌ ŋ k iː
	 r ɪ ˈ l ɪ k s
	 ˌ p iː ˌ ɛ g ɪ ŋ
	 p ə ˈ m iː
	 ˈ t ɛ d oʊ
Epoch 293: train loss: 0.5495	dev loss: 0.8012                                                                                               
	Generated: in train: 10%, assess: 0%, novel: 90%
	 k æ ˈ ʒ ɑː r ə m z
	 ɪ ˈ r ɛ n dʒ ɪ d
	 ˈ g ʌ n h ə l
	 ˈ k r ɛ l ɪ d
	 ˈ ɑː r b ɝ ˌ d ɝː d
Epoch 294: train loss: 0.5495	dev loss: 0.8011                                                                                               
	Generated: in train: 10%, assess: 3%, novel: 87%
	 ˈ b æ v ɪ ŋ
	 ˈ v ɑː l r z ə n
	 ˈ p oʊ ˌ t ɪ n d
	 ˈ h ɔ r m i
	 ˈ æ n ɝ ˌ ɑː l ɪ dʒ
Epoch 295: train loss: 0.5494	dev loss: 0.8010                                                                                               
	Generated: in train: 7

Epoch 321: train loss: 0.5487	dev loss: 0.8000                                                                                               
	Generated: in train: 13%, assess: 1%, novel: 86%
	 ˈ b aɪ ˌ k eɪ
	 ˈ r ʌ k ə
	 ˈ w ɪ ŋ g ə l
	 g ɪ ˈ l aɪ d
	 k ɪ n k ɝ ˈ s n ɪ s
Epoch 322: train loss: 0.5487	dev loss: 0.8000                                                                                               
	Generated: in train: 14%, assess: 0%, novel: 86%
	 ˈ b æ s t ə l s k i
	 ˈ s t ɛ v ɝ b iː d
	 ˈ l aʊ t r ə z
	 ˈ s ɛ r ˌ dʒ eɪ t
	 ˈ m ɪ ŋ g r ə d
Epoch 323: train loss: 0.5486	dev loss: 0.7999                                                                                               
	Generated: in train: 14%, assess: 0%, novel: 86%
	 ˈ t ɔ r ə l z
	 ˈ m æ l i
	 ə ˈ k oʊ l ɔ z
	 ˈ s ʌ n ɝ
	 ˈ s ɛ s d
Epoch 324: train loss: 0.5487	dev loss: 0.8000                                                                                               
	Generated: in train: 6%, assess: 

Epoch 350: train loss: 0.5480	dev loss: 0.7991                                                                                               
	Generated: in train: 9%, assess: 0%, novel: 91%
	 ˈ oʊ l ɪ t
	 ˌ s ɛ t ə ˈ m ɝː n oʊ
	 ˌ f r æ m aɪ ˈ k j uː z iː ə l
	 ˈ s ɪ ŋ g ə l ˌ m eɪ t s
	 ˈ r ɛ s t b i
Epoch 351: train loss: 0.5480	dev loss: 0.7990                                                                                               
	Generated: in train: 8%, assess: 2%, novel: 90%
	 ɪ n ˈ k w aɪ d
	 ˈ d ɑː l ɪ n
	 ˈ d ɑː r g ə n v
	 ˈ f r ɪ n ə m
	 ə ˈ n æ s t ə
Epoch 352: train loss: 0.5480	dev loss: 0.7990                                                                                               
	Generated: in train: 8%, assess: 0%, novel: 92%
	 r iː b ɝ ˈ ɪ n
	 ˈ aʊ ˌ s t ɑː d
	 ˈ b ɔ r z
	 ˈ b r æ n z l ə
	 ˌ h j uː ɝ ˈ iː t
Epoch 353: train loss: 0.5479	dev loss: 0.7990                                                                                               
	Gen

Epoch 379: train loss: 0.5475	dev loss: 0.7983                                                                                               
	Generated: in train: 14%, assess: 2%, novel: 84%
	 ˈ d ɔ r tʃ oʊ z
	 ˈ g r eɪ s ɝ ˌ s ɛ s k ɪ s
	 n uː ˈ m ɔ n
	 k ɝ ɝ ˈ t ɪ ʃ ə n s
	 n uː ˈ ɑː r v ə n t s
Epoch 380: train loss: 0.5474	dev loss: 0.7983                                                                                               
	Generated: in train: 7%, assess: 1%, novel: 92%
	 ˈ w eɪ m oʊ
	 ˈ ɑː r n ɪ θ t ɝ
	 ˈ s k ɑː r k i
	 s ɑː ˈ θ ɑː r ʃ ə n eɪ
	 ˈ
Epoch 381: train loss: 0.5475	dev loss: 0.7984                                                                                               
	Generated: in train: 13%, assess: 0%, novel: 87%
	 b ɑː ˈ dʒ ɪ t i
	 ˈ v ɔ r ə ˌ w ɪ ʒ i
	 ˈ ʃ ɝː k m ə n
	 ˈ b ɑː k t i
	 ˈ s ɔ g ə z l i
Epoch 382: train loss: 0.5474	dev loss: 0.7982                                                                                               
	Gener

Early stopping because of no decrease in 3 epochs.


In [28]:
%%time

model_parameters = ModelParams(
    rnn_type='gru', num_layers=1, embedding_dimension=10, hidden_dimension=5,
    max_epochs=1000, early_stopping_rounds=3
)

model_10_5_1 = LanguageModel(vocab, model_parameters, device_name='cuda')

_ = model_10_5_1.fit(
    train_pronunciations.pronunciation.values.tolist(),
    dev_pronunciations.pronunciation.values.tolist(),
    print_every=1,
)

with open('model_10_5_1.pt', 'wb') as fh:
    model_10_5_1.save(fh)

Epoch 1: train loss: 2.8791	dev loss: 3.0447                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 θ r ɔɪ ʃ j ˈ
	 uː
	 v ɔɪ v tʃ ŋ ɛ ɛ f æ ʊ ɝ z ˌ d g ʃ aʊ θ t s ð
	 h ɔɪ ʌ
	 ɪ ɝ v tʃ oʊ ɛ b
Epoch 2: train loss: 1.9111	dev loss: 2.2106                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 ɑː
	 b ð ɔɪ
	 ɝ ʊ g ʃ ɔ ˈ aʊ ɝː iː ʌ tʃ ɑː f uː uː ʒ aɪ uː ɔɪ iː r k ɝː iː k l æ dʒ oʊ
	 f ˌ g uː w v ɝ ŋ n b ə p
	 ɝː ɛ p j p ɪ d ŋ n ɔɪ iː uː n θ ɝː t ʌ g v ɝ tʃ oʊ j n iː uː ɑː tʃ
Epoch 3: train loss: 1.3768	dev loss: 1.7297                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 ŋ ɪ k ɝː ɛ b æ ʊ ɝː g s aɪ
	 m ɑː ð æ ɛ uː l p ə ɔ w uː ɔɪ d w v ð ɔɪ
	 uː ɝː ŋ z b
	 ɪ ɔɪ z oʊ ɛ m aʊ eɪ ɔɪ aʊ ð ɛ ɛ dʒ ə
	 k ɔɪ i aɪ
Epoch 4: train

Epoch 30: train loss: 0.6463	dev loss: 0.9379                                                                                               
	Generated: in train: 12%, assess: 1%, novel: 87%
	 ˈ l eɪ l
	 ˈ r aɪ ˌ n iː
	 ˈ s ɛ t ɔ t uː
	 m oʊ ə k ə ˈ dʒ ɛ f m aɪ d ɪ l
	 ˈ b iː t i
Epoch 31: train loss: 0.6434	dev loss: 0.9340                                                                                               
	Generated: in train: 9%, assess: 2%, novel: 89%
	 ŋ ˈ s s aʊ g ŋ
	 l ɪ ˈ w æ n s
	 ˈ v ɛ p s
	 ˈ n z ɛ r
	 ˈ k ʌ s l aɪ
Epoch 32: train loss: 0.6406	dev loss: 0.9301                                                                                               
	Generated: in train: 7%, assess: 1%, novel: 92%
	 ˌ l r k r iː f ɑː ˈ z b ʌ n b l ɪ z ˌ b ə b j ɝ
	 ˌ r iː z n ə ˈ l ɪ tʃ i
	 ˈ t ɛ t d b m eɪ n
	 n ˈ ɑː n n r ə aɪ p ɝ
	 ˈ k ɛ d ɛ m t
Epoch 33: train loss: 0.6382	dev loss: 0.9269                                                                                     

Epoch 59: train loss: 0.6062	dev loss: 0.8825                                                                                               
	Generated: in train: 11%, assess: 0%, novel: 89%
	 ˈ r ɝ b ɪ s iː ɝ
	 ˈ g p ɪ b uː
	 f ɪ ə ˈ p æ l r
	 ˈ r ʌ s l i ɝ s z
	 l ɑː l ˈ p ɑː r i
Epoch 60: train loss: 0.6055	dev loss: 0.8814                                                                                               
	Generated: in train: 10%, assess: 1%, novel: 89%
	 ˈ d æ n ɪ ŋ
	 ˈ tʃ ɛ p ɝ
	 ˈ s t ɪ n ə n
	 ˈ s æ l ə v
	 ˈ k l eɪ z
Epoch 61: train loss: 0.6048	dev loss: 0.8805                                                                                               
	Generated: in train: 9%, assess: 1%, novel: 90%
	 r ˈ w ɔɪ l
	 iː f ə ˈ m ɑː s
	 ɪ ŋ ˈ p ɪ l ɑː
	 ˈ ɑː n z ɪ f t
	 ˈ h æ k ə
Epoch 62: train loss: 0.6041	dev loss: 0.8796                                                                                               
	Generated: in train: 17%, assess: 0%, novel: 83

Epoch 89: train loss: 0.5949	dev loss: 0.8665                                                                                               
	Generated: in train: 6%, assess: 1%, novel: 93%
	 ˈ n ɝː r ɔ n
	 ˈ l ɑː t ə t
	 ˈ s l ɑː ʃ ə n ɑː n
	 ɪ n ˈ k ɛ p
	 ɑː ʃ ə ˈ m ɝː t ə l
Epoch 90: train loss: 0.5948	dev loss: 0.8664                                                                                               
	Generated: in train: 4%, assess: 2%, novel: 94%
	 ˈ m ɛ l ə k d t i
	 d ɑː v ˈ l ɛ r l ə n
	 ə ˈ t ɪ n ˌ t uː f ɪ n
	 ɝ ˈ k w ɔ r i
	 ˈ s ʌ g r ə f ɪ t g
Epoch 91: train loss: 0.5945	dev loss: 0.8660                                                                                               
	Generated: in train: 16%, assess: 2%, novel: 82%
	 ˈ r uː l t ə θ
	 ˈ ɛ k k ɪ t
	 ɝ ˈ w æ n z l ɪ ŋ
	 n ˈ k ʌ n ɝ
	 ˈ f r ʌ ŋ r oʊ
Epoch 92: train loss: 0.5944	dev loss: 0.8658                                                                                               
	Generated: 

Epoch 118: train loss: 0.5913	dev loss: 0.8615                                                                                               
	Generated: in train: 12%, assess: 0%, novel: 88%
	 ˈ f ˌ h iː s ə k ɝ ˌ ŋ l ɪ t
	 ˈ b ɛ r oʊ
	 ˈ f ɑː r k ə n
	 ˈ j ɑː ˌ s z ə z l iː z n
	 ˈ p t ɑː r ɝ p
Epoch 119: train loss: 0.5912	dev loss: 0.8614                                                                                               
	Generated: in train: 17%, assess: 2%, novel: 81%
	 ˈ p ɑː r
	 iː ə ˈ s r ɪ n r ɑː p ɝ z
	 ˈ w ɪ l ɝ
	 ˈ k æ d p ɪ s
	 l ə ˈ r ɑː r ə
Epoch 120: train loss: 0.5911	dev loss: 0.8613                                                                                               
	Generated: in train: 14%, assess: 3%, novel: 83%
	 ˈ r æ n t i
	 z d oʊ ˈ r eɪ b ə z
	 ˈ dʒ iː v t
	 ˈ æ l ɝ ɛ ŋ
	 r ɪ ŋ ˈ g ɛ r ə v i
Epoch 121: train loss: 0.5911	dev loss: 0.8612                                                                                               
	Gener

...

Epoch 293: train loss: 0.5795	dev loss: 0.8442                                                                                               
	Generated: in train: 12%, assess: 0%, novel: 88%
	 ˈ m eɪ t l ɝ ˌ t r eɪ z
	 s uː ˈ b eɪ t ə ŋ
	 z iː ˈ k ɛ l l ɪ t
	 ˈ t n iː ˌ f ɪ t
	 ˈ w w aɪ d ɝ k ˌ s t l eɪ b ɝ z
Epoch 294: train loss: 0.5794	dev loss: 0.8441                                                                                               
	Generated: in train: 10%, assess: 0%, novel: 90%
	 ˈ d æ d i
	 ˌ s ɑː n m aʊ ˈ ʃ aɪ d ə f n
	 ˈ s t r ɛ t ə n iː ə t
	 ˈ k r ɛ m m d iː ɝ
	 dʒ ɪ m ˈ tʃ ɔ r ə n n
Epoch 295: train loss: 0.5794	dev loss: 0.8441                                                                                               
	Generated: in train: 10%, assess: 1%, novel: 89%
	 ˈ s ɑː ˌ s t ɑː ˌ l iː s i
	 ˈ w ɪ p ə l z i
	 ˈ g l uː d ɝ z
	 ˈ s w aʊ t r
	 ˈ s ɪ ˌ n æ n n ɝ z
Epoch 296: train loss: 0.5794	dev loss: 0.8441                                            

Epoch 322: train loss: 0.5791	dev loss: 0.8437                                                                                               
	Generated: in train: 6%, assess: 2%, novel: 92%
	 ˈ l ɛ l ɪ g t
	 ˈ z aɪ t ə l
	 ˈ r æ l ɪ ŋ ˌ s æ v i
	 ˈ j uː d z ɝ ˌ oʊ ˌ t aɪ n z
	 ˈ p j ɝː r ˌ ə l ə l t ə n d
Epoch 323: train loss: 0.5790	dev loss: 0.8436                                                                                               
	Generated: in train: 11%, assess: 1%, novel: 88%
	 ˈ p ɛ l t r iː ˌ d eɪ n ɪ n
	 m ɪ ˈ z uː l ɪ ˌ t æ k
	 ˈ n ɪ ŋ t ɪ ŋ
	 ˈ d ɪ n s ə
	 ˈ k t r aʊ d
Epoch 324: train loss: 0.5791	dev loss: 0.8436                                                                                               
	Generated: in train: 6%, assess: 4%, novel: 90%
	 m ɪ ˈ g ɪ θ ə
	 ˈ j ɑː l θ
	 ˈ s t ɔ s i
	 s oʊ ˈ f æ k ɪ p
	 ˈ w æ n n ə s z
Epoch 325: train loss: 0.5791	dev loss: 0.8436                                                                                   

Epoch 351: train loss: 0.5788	dev loss: 0.8432                                                                                               
	Generated: in train: 11%, assess: 2%, novel: 87%
	 ˈ k iː r k
	 ˈ g ɔ r ɝ z
	 ˈ g oʊ z
	 ˈ ɑː r ɪ dʒ ɪ m ə n
	 ˈ g r ɪ n z ɪ ŋ
Epoch 352: train loss: 0.5788	dev loss: 0.8432                                                                                               
	Generated: in train: 14%, assess: 0%, novel: 86%
	 ˈ l aɪ ˌ s eɪ k ə n
	 j ə ˈ m ɪ k ɝ z
	 tʃ ɪ s ˈ p ɑː l s
	 ˈ w iː l ə m
	 p ɝ ˈ h æ ŋ b ɑː l
Epoch 353: train loss: 0.5788	dev loss: 0.8432                                                                                               
	Generated: in train: 5%, assess: 2%, novel: 93%
	 ˌ v ɑː r p ə k ˈ p r ʌ m l
	 ˈ p ɑː l ˌ h ɔɪ z
	 ˈ r aʊ
	 ˈ p r iː r ɪ d
	 ˈ v eɪ t l ə
Epoch 354: train loss: 0.5788	dev loss: 0.8432                                                                                               
	Generated: in trai

Epoch 380: train loss: 0.5785	dev loss: 0.8429                                                                                               
	Generated: in train: 6%, assess: 2%, novel: 92%
	 iː ˈ r ɛ l d
	 ˌ ə r ə ˈ p ɪ n ʃ ə n d
	 ˌ p r aʊ ə n s ɪ ˈ s ɔ t d
	 ˈ p ɝː r ə ˌ m ɪ l
	 ˈ l ɔ t ə n t
Epoch 381: train loss: 0.5785	dev loss: 0.8429                                                                                               
	Generated: in train: 5%, assess: 3%, novel: 92%
	 ˈ k j ɛ r v oʊ t
	 ˈ ʃ ɝː k p r ɝ t
	 l ə ˈ d uː s z ɪ b
	 aɪ ˈ s ɝː s t
	 ˈ h uː ɝ
Epoch 382: train loss: 0.5785	dev loss: 0.8428                                                                                               
	Generated: in train: 12%, assess: 1%, novel: 87%
	 ˈ ɪ k ɝ ˌ ɑː t ə ˌ t eɪ z
	 ˈ l uː d
	 ˈ p ɑː n ɪ d
	 ˈ d r uː ə k z
	 ˈ k ɑː l t s
Epoch 383: train loss: 0.5786	dev loss: 0.8429                                                                                               
	Gene

Early stopping because of no decrease in 3 epochs.


In [29]:
%%time

model_parameters = ModelParams(
    rnn_type='gru', num_layers=1, embedding_dimension=10, hidden_dimension=3,
    max_epochs=1000, early_stopping_rounds=3
)

model_10_3_1 = LanguageModel(vocab, model_parameters, device_name='cuda')

_ = model_10_3_1.fit(
    train_pronunciations.pronunciation.values.tolist(),
    dev_pronunciations.pronunciation.values.tolist(),
    print_every=1,
)

with open('model_10_3_1.pt', 'wb') as fh:
    model_10_3_1.save(fh)

Epoch 1: train loss: 2.9317	dev loss: 3.0980                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 ɝ m ˌ z ɝː ɔ ð uː ˈ ɝ k ɔ ɛ aɪ ə r
	 aɪ h
	 ɝː ʒ ɝ s aɪ j p oʊ ð l l ɝ j z
	 v ɔ k
	 i
Epoch 2: train loss: 2.2159	dev loss: 2.4811                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 f t t d ˌ ʒ ˌ dʒ ˈ ˌ ˈ iː l s
	 i ŋ l ɝː v uː
	 b n ɔ tʃ ʒ
	 uː ɝ oʊ ʌ ɛ d i aʊ z
	 ð æ j i ʌ m j ɔ
Epoch 3: train loss: 1.7110	dev loss: 2.0369                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 eɪ ˈ l i ɪ b uː ð ɛ æ k f ʌ ˈ m iː w r p ə ɔɪ ə tʃ
	 eɪ æ r
	 iː ð l
	 ʌ aɪ
	 aɪ
Epoch 4: train loss: 1.4059	dev loss: 1.7616                                                                                               
	Generated: in t

Epoch 31: train loss: 0.7108	dev loss: 1.0278                                                                                               
	Generated: in train: 1%, assess: 0%, novel: 99%
	 ʃ ə ə ˌ l ɛ ə v
	 w
	 g ə ˈ j s m
	 ˈ k iː r ɑː n z
	 ˈ h b
Epoch 32: train loss: 0.7066	dev loss: 1.0221                                                                                               
	Generated: in train: 3%, assess: 0%, novel: 97%
	 ˈ k oʊ d ʃ ˌ ɝ b ŋ ˌ l s ɛ ɪ l n
	 ˈ d ɛ aʊ ɝ ɝ
	 ˈ s s ə l ə k n
	 ˈ ŋ ə v n
	 ˌ uː uː ˈ n l ə ˈ ʃ ɪ æ ʒ
Epoch 33: train loss: 0.7026	dev loss: 1.0166                                                                                               
	Generated: in train: 1%, assess: 0%, novel: 99%
	 ˌ b aʊ ˈ ɑː æ n ɪ ŋ
	 
	 ˈ s ɪ s ɪ ɪ n
	 
	 ˈ v eɪ n f
Epoch 34: train loss: 0.6988	dev loss: 1.0115                                                                                               
	Generated: in train: 2%, assess: 0%, novel: 98%
	 ˌ d
	 ˈ s ɑ

Epoch 61: train loss: 0.6487	dev loss: 0.9423                                                                                               
	Generated: in train: 2%, assess: 0%, novel: 98%
	 ˈ k uː k k w iː n ɪ r
	 n
	 ˈ ʌ k ɝ l oʊ t
	 ˈ p uː n ɝ ɪ v t i z
	 iː ˈ t iː
Epoch 62: train loss: 0.6478	dev loss: 0.9411                                                                                               
	Generated: in train: 0%, assess: 1%, novel: 99%
	 ˈ r t v ɪ t l ə d ˈ m z ɝ t
	 r ˈ h iː t ɪ l n
	 ˈ d d s ɔɪ ɑː s ˌ ɑː m p t ɝ iː
	 ˈ oʊ ˈ b
	 ˈ f ɔ s r ɪ
Epoch 63: train loss: 0.6470	dev loss: 0.9400                                                                                               
	Generated: in train: 3%, assess: 2%, novel: 95%
	 ˈ oʊ z
	 
	 ə ˈ n ɔ ɪ z
	 ˈ h ɑː n t ɪ n
	 ˈ s k iː v s
Epoch 64: train loss: 0.6462	dev loss: 0.9390                                                                                               
	Generated: in train: 8%, assess: 0%, novel

Epoch 90: train loss: 0.6317	dev loss: 0.9191                                                                                               
	Generated: in train: 7%, assess: 1%, novel: 92%
	 ˈ l æ v ə ˌ t ɝ t ɝ r ə s oʊ
	 ˈ ɔ n ə z
	 ˈ k w eɪ m ɪ m ɑː ˌ ɪ n z ə d n ɪ z
	 ˈ g ɛ n t ɝ k i
	 z ˈ s iː k i
Epoch 91: train loss: 0.6313	dev loss: 0.9184                                                                                               
	Generated: in train: 8%, assess: 0%, novel: 92%
	 ˈ m ɑː ʃ iː b ɝ
	 ˈ ɑː r s ə m
	 oʊ ˈ f oʊ m ɛ n ˌ ʃ ɝ
	 ˌ ˈ f ə t
	 ɝ ˈ k ɪ f ə l
Epoch 92: train loss: 0.6305	dev loss: 0.9173                                                                                               
	Generated: in train: 7%, assess: 1%, novel: 92%
	 
	 ˌ ˈ p s l ʊ n
	 ˈ p uː g
	 d dʒ
	 ˈ l iː r t iː z
Epoch 93: train loss: 0.6299	dev loss: 0.9165                                                                                               
	Generated: in train: 8%, assess: 2

Epoch 120: train loss: 0.6161	dev loss: 0.8968                                                                                               
	Generated: in train: 9%, assess: 0%, novel: 91%
	 p oʊ ˈ eɪ l ɝ t ɝ d z
	 dʒ ɝ ˈ w uː n
	 ˈ ɑː t i
	 ˈ m t ʊ
	 ˈ p p ɔ k z ɝ
Epoch 121: train loss: 0.6159	dev loss: 0.8966                                                                                               
	Generated: in train: 3%, assess: 0%, novel: 97%
	 ˈ t ə n ə t n ɪ dʒ iː
	 ˌ s eɪ p aɪ l ə ˈ r æ p
	 ˈ n ɑː k r ɪ z
	 ˈ r ɑː ˌ t ɑː ŋ r j ə ˈ tʃ
	 m ɪ ˈ t uː ˌ r oʊ
Epoch 122: train loss: 0.6157	dev loss: 0.8962                                                                                               
	Generated: in train: 7%, assess: 1%, novel: 92%
	 ˈ r r oʊ n n t ə n ˌ ʃ ə l
	 ˈ r ɛ l ɪ s
	 n ə ˈ ʃ oʊ b k ə t iː
	 n ə ˈ n ɑː f z ə ŋ
	 ˈ w aɪ s
Epoch 123: train loss: 0.6156	dev loss: 0.8961                                                                                         

...

Epoch 208: train loss: 0.6079	dev loss: 0.8855                                                                                               
	Generated: in train: 9%, assess: 0%, novel: 91%
	 t oʊ ˈ m aɪ tʃ ɝ
	 ˈ p eɪ z
	 s ˈ s t r iː l ɪ
	 ˌ v æ p ˈ h ɛ t ɝ
	 ˈ l ɔɪ n iː n ə l
Epoch 209: train loss: 0.6078	dev loss: 0.8853                                                                                               
	Generated: in train: 8%, assess: 1%, novel: 91%
	 ˈ t eɪ k g t
	 ˈ l iː s
	 ˈ b eɪ d
	 ə ˈ t æ m ˌ l aɪ n ɝ
	 s ˈ f ɝː r ˌ ɝː d
Epoch 210: train loss: 0.6077	dev loss: 0.8853                                                                                               
	Generated: in train: 8%, assess: 0%, novel: 92%
	 m ˈ f oʊ r ə n
	 ˈ s ɝː t ɪ n
	 ˈ s p ɪ n z
	 ˈ n ɑː f ə n ˌ ɪ r z
	 ə n n ˈ t iː n ə
Epoch 211: train loss: 0.6077	dev loss: 0.8853                                                                                               
	Generated: in train: 9%, as

Epoch 238: train loss: 0.6070	dev loss: 0.8842                                                                                               
	Generated: in train: 11%, assess: 1%, novel: 88%
	 l oʊ ˈ k æ n ˌ ɔ k t ɪ l
	 ˈ h æ s i
	 ə ˈ f iː z
	 d ɪ ˈ b ɪ d z
	 t ɪ s p ɑː r ɪ ˈ t ə n iː
Epoch 239: train loss: 0.6070	dev loss: 0.8843                                                                                               
	Generated: in train: 15%, assess: 2%, novel: 83%
	 ˈ p ɛ f ɝ l ə n n ə s
	 ˈ ɔ s
	 ˈ k æ t ə m ɪ n
	 ˈ ɛ r ɪ ŋ
	 ˈ n æ p t s
Epoch 240: train loss: 0.6070	dev loss: 0.8843                                                                                               
	Generated: in train: 8%, assess: 1%, novel: 91%
	 ˈ g uː z ə z
	 ɪ ˈ l ə n aɪ
	 ə ˈ h ɔ ʃ ɪ m g
	 ˈ r ɑː n n iː
	 ˈ l aɪ
Epoch 241: train loss: 0.6069	dev loss: 0.8842                                                                                               
	Generated: in train: 7%, assess: 2%, 

Epoch 267: train loss: 0.6066	dev loss: 0.8836                                                                                               
	Generated: in train: 5%, assess: 1%, novel: 94%
	 ə ˈ s aʊ n eɪ k d
	 ˈ h ɑː n ə
	 v r æ f ˈ ɛ s t i
	 ˈ s w ʌ n ɝ k ɪ k n ɝ t ə g p ˌ r uː
	 ˈ k ɔ l d ɪ ŋ
Epoch 268: train loss: 0.6066	dev loss: 0.8837                                                                                               
	Generated: in train: 7%, assess: 1%, novel: 92%
	 ɪ n ˈ f ɛ k w eɪ ˌ s r ʌ t s
	 ˈ h ɑː p t
	 g ə ˈ h eɪ h p ɪ θ l i
	 ˈ h æ n l iː ɑː t
	 ˈ d r oʊ t
Epoch 269: train loss: 0.6066	dev loss: 0.8837                                                                                               
	Generated: in train: 11%, assess: 1%, novel: 88%
	 ˈ r iː t r ə m i d
	 p ə ˈ p ɔ s aɪ s
	 ˈ dʒ ɪ dʒ
	 ˈ d ɝː k dʒ ɪ d
	 r ə ˈ r m ɪ ˌ s aɪ k r i
Epoch 270: train loss: 0.6065	dev loss: 0.8836                                                                         

Epoch 296: train loss: 0.6063	dev loss: 0.8833                                                                                               
	Generated: in train: 5%, assess: 1%, novel: 94%
	 ˈ ɑː ˌ s l i
	 ɪ ˈ t b ɑː p ə l k
	 i
	 b ɪ n ˌ ɪ m d æ l ˈ p aʊ s ɪ n t ə z
	 f iː ˈ d ɛ
Epoch 297: train loss: 0.6064	dev loss: 0.8833                                                                                               
	Generated: in train: 12%, assess: 1%, novel: 87%
	 ˈ l aɪ l ɝ t ə l
	 ˈ l ʌ ˌ ʃ ɪ n l
	 p ˈ k w aɪ s z
	 tʃ ə ˈ ɑː b ɪ
	 ˈ b ɝ t ɔ h aɪ n
Epoch 298: train loss: 0.6063	dev loss: 0.8833                                                                                               
	Generated: in train: 13%, assess: 2%, novel: 85%
	 p oʊ ˈ n ɑː b ɪ m ə n
	 ˈ d t w eɪ h ɪ l
	 ˈ s ɝː k b oʊ
	 ˈ n ə t l ɝ
	 ˈ n ɛ t
Epoch 299: train loss: 0.6063	dev loss: 0.8832                                                                                               
	Generated: in trai

Early stopping because of no decrease in 3 epochs.


In [30]:
%%time

model_parameters = ModelParams(
    rnn_type='rnn', num_layers=1, embedding_dimension=10, hidden_dimension=3,
    max_epochs=1000, early_stopping_rounds=3
)

rnn_10_3_1 = LanguageModel(vocab, model_parameters, device_name='cuda')

_ = rnn_10_3_1.fit(
    train_pronunciations.pronunciation.values.tolist(),
    dev_pronunciations.pronunciation.values.tolist(),
    print_every=1,
)

with open('rnn_10_3_1.pt', 'wb') as fh:
    rnn_10_3_1.save(fh)

Epoch 1: train loss: 3.0106	dev loss: 3.1681                                                                                               
	Generated: in train: 1%, assess: 0%, novel: 99%
	 oʊ ɝː ʃ w i ʃ t v ʒ ɔ tʃ ɔɪ æ ʃ ɔ m w ɝ b l ɛ s ʌ h f s
	 iː ʊ
	 s i ɑː eɪ ð w ʌ k dʒ ɪ dʒ k r ɝː b k v iː w f ˈ tʃ d ɔ ɔ tʃ ɔɪ g k ʊ ɑː ʃ n m b ɔɪ ˌ ˈ aɪ ɝː uː m tʃ r ɔ eɪ
	 w g iː t iː n uː ɔ ɝː
	 s dʒ
Epoch 2: train loss: 2.3713	dev loss: 2.6146                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 g z
	 ˈ ɔɪ ɔ k
	 
	 
	 eɪ aʊ iː ð ɔɪ
Epoch 3: train loss: 1.8933	dev loss: 2.1995                                                                                               
	Generated: in train: 0%, assess: 0%, novel: 100%
	 z aʊ r oʊ ɛ m s
	 ʃ ɑː oʊ s s
	 ʊ
	 θ aʊ m
	 tʃ ʒ uː
Epoch 4: train loss: 1.5562	dev loss: 1.8990                                                                                       

Epoch 31: train loss: 0.6963	dev loss: 1.0061                                                                                               
	Generated: in train: 8%, assess: 0%, novel: 92%
	 ˈ k iː r
	 d b ə
	 ˈ ʒ ɑː ˌ uː f dʒ ɪ h j ɪ d
	 ˈ f ɔɪ s k ɛ n t t z ə ɛ ˈ ʃ ə ʃ r
	 ˈ h ɔ ʒ ə oʊ ˈ s ɑː ˈ
Epoch 32: train loss: 0.6915	dev loss: 0.9996                                                                                               
	Generated: in train: 3%, assess: 1%, novel: 96%
	 ˈ
	 ˈ tʃ ʌ ŋ z aɪ
	 
	 ˈ dʒ ɔɪ æ t
	 
Epoch 33: train loss: 0.6868	dev loss: 0.9932                                                                                               
	Generated: in train: 4%, assess: 0%, novel: 96%
	 
	 ˈ æ ˈ p ə n ɔ t i ə d ə k
	 l ɔɪ ɔ
	 ʌ s ŋ
	 ˈ h ɛ k ɪ t ə z
Epoch 34: train loss: 0.6825	dev loss: 0.9873                                                                                               
	Generated: in train: 6%, assess: 0%, novel: 94%
	 ˈ ʃ
	 ˈ k dʒ ɑː h z æ ʃ

Epoch 61: train loss: 0.6411	dev loss: 0.9317                                                                                               
	Generated: in train: 13%, assess: 2%, novel: 85%
	 n
	 ˈ m æ g z i
	 n m
	 ˈ b b eɪ l
	 ˈ k ɪ s ə l
Epoch 62: train loss: 0.6400	dev loss: 0.9302                                                                                               
	Generated: in train: 12%, assess: 1%, novel: 87%
	 ˈ æ n s
	 ˈ ʒ ɑː n k
	 ˈ ɪ n ɪ ʃ
	 ˈ m v ɔ d
	 ˌ ʊ l
Epoch 63: train loss: 0.6388	dev loss: 0.9286                                                                                               
	Generated: in train: 13%, assess: 0%, novel: 87%
	 ˈ p ɔ s
	 ˌ g ɪ s l ə d
	 ˌ w ɛ k
	 ˈ b ɑː s ʃ ə d
	 n i
Epoch 64: train loss: 0.6378	dev loss: 0.9271                                                                                               
	Generated: in train: 7%, assess: 1%, novel: 92%
	 ˈ
	 ˈ p
	 n
	 ˈ n ɝː n t i ɪ ˈ n ɛ b ɪ d
	 ˈ s ʌ n z
Epoch 65: train l

Epoch 92: train loss: 0.6183	dev loss: 0.9002                                                                                               
	Generated: in train: 15%, assess: 3%, novel: 82%
	 ˈ s ɑː z
	 ˈ v ɪ b aɪ r z
	 ˈ s ɪ t
	 ˈ s v iː
	 ˈ f uː s
Epoch 93: train loss: 0.6178	dev loss: 0.8995                                                                                               
	Generated: in train: 12%, assess: 3%, novel: 85%
	 m ə ˈ ɛ n ə s l ɪ n d n
	 ˈ h iː l z
	 ˈ n r ʊ n s
	 ˈ k t b uː ˌ k ɪ n
	 ˈ r m ɪ m ɪ n m
Epoch 94: train loss: 0.6173	dev loss: 0.8988                                                                                               
	Generated: in train: 8%, assess: 2%, novel: 90%
	 r ˈ b ɛ z ɝ
	 ˈ m s ə b t oʊ
	 ˈ s n eɪ f
	 ˈ ɛ ˌ æ ŋ t uː z n
	 ˈ w æ ˌ dʒ aɪ ˈ b ɛ ð ɪ n ə ˌ l æ tʃ
Epoch 95: train loss: 0.6169	dev loss: 0.8981                                                                                               
	Generated: in train: 16%, asse

Epoch 122: train loss: 0.6086	dev loss: 0.8866                                                                                               
	Generated: in train: 11%, assess: 3%, novel: 86%
	 ˈ r ɪ n ə l t
	 r ɪ ˈ n t ɛ dʒ ə n ˌ r eɪ g
	 ˈ n ɝː l
	 ˌ r ɑː g ɪ d ˈ aʊ t ɝ r
	 ˈ p æ h ɪ l ə d
Epoch 123: train loss: 0.6085	dev loss: 0.8864                                                                                               
	Generated: in train: 17%, assess: 1%, novel: 82%
	 
	 ˌ p æ w iː ˈ r d ʌ s
	 ˈ oʊ tʃ ɝ ə d
	 ˈ ɝː h ə l
	 s ə ˈ t oʊ b l ɝ
Epoch 124: train loss: 0.6083	dev loss: 0.8862                                                                                               
	Generated: in train: 5%, assess: 1%, novel: 94%
	 ˌ k tʃ ɑː ˈ p ɛ l ə s ˌ p eɪ m
	 ˈ s æ s m ə l
	 ˈ g ʌ z
	 ˈ n ɔ d ɝ
	 ˈ s s iː b ə ˌ m aɪ z
Epoch 125: train loss: 0.6082	dev loss: 0.8860                                                                                               
	Generated: i

Epoch 151: train loss: 0.6058	dev loss: 0.8827                                                                                               
	Generated: in train: 6%, assess: 0%, novel: 94%
	 ˈ t aʊ r ɪ n ɝ
	 ˈ f uː r ɪ ŋ ˌ s ɪ m
	 l ɛ ˈ k ʌ r t s i
	 ə ˈ l æ d ɪ l
	 r ɑː ˌ n ɪ t ˈ v ɑː t ˌ n ɪ n s t
Epoch 152: train loss: 0.6057	dev loss: 0.8825                                                                                               
	Generated: in train: 8%, assess: 1%, novel: 91%
	 ˈ ʌ l ɪ n
	 ˈ k eɪ d
	 ˈ k ɔ n s oʊ
	 ˈ h iː r
	 ˈ w ɛ b ə t m
Epoch 153: train loss: 0.6056	dev loss: 0.8824                                                                                               
	Generated: in train: 10%, assess: 1%, novel: 89%
	 ˌ oʊ s r ɪ n ə ˈ s w eɪ p
	 m ə ˈ p ɑː l n ə t ɝ
	 m iː ˈ f ʌ s k ɝ
	 ˈ f ɛ r s l ɪ ŋ t t ə s
	 ˈ θ m ɛ v ɪ n ɝ
Epoch 154: train loss: 0.6057	dev loss: 0.8826                                                                                         

Epoch 180: train loss: 0.6047	dev loss: 0.8811                                                                                               
	Generated: in train: 16%, assess: 3%, novel: 81%
	 ˈ n w ɪ t
	 ˈ v ɪ n oʊ
	 ˈ z aɪ l oʊ ə k
	 t iː ˌ ɛ k ɪ n ˈ ɝː g i
	 ˈ dʒ ɑː d
Epoch 181: train loss: 0.6046	dev loss: 0.8809                                                                                               
	Generated: in train: 9%, assess: 0%, novel: 91%
	 ˈ p ɑː n ɪ n æ ŋ
	 ˌ t ɛ n ə f ˈ ʃ ɛ n
	 ˈ v aɪ p d ɝ
	 ˈ r ɝː k ɝ
	 ˈ s æ l iː
Epoch 182: train loss: 0.6045	dev loss: 0.8808                                                                                               
	Generated: in train: 10%, assess: 1%, novel: 89%
	 ˈ d eɪ d
	 ˈ s s æ f l
	 k ə ˈ g æ n r t ɪ ŋ
	 ˈ ɔ r ə ʃ
	 m ɪ ˈ b iː z
Epoch 183: train loss: 0.6044	dev loss: 0.8807                                                                                               
	Generated: in train: 7%, assess: 1%, novel: 9

Epoch 209: train loss: 0.6038	dev loss: 0.8798                                                                                               
	Generated: in train: 10%, assess: 0%, novel: 90%
	 ˈ ɝː r t r ə n n s
	 ˈ b m uː z
	 ˈ s ɛ l
	 ˈ s s r æ n d ɝ
	 m ˈ p w ʌ ŋ
Epoch 210: train loss: 0.6038	dev loss: 0.8797                                                                                               
	Generated: in train: 8%, assess: 1%, novel: 91%
	 ˈ p ɛ m ɪ n ˌ oʊ s
	 ə ˈ d oʊ r ə ˌ s ɔɪ
	 r ə b i
	 ˌ b ɑː l i
	 ˈ l ɑː v ɝ ˌ k iː
Epoch 211: train loss: 0.6037	dev loss: 0.8797                                                                                               
	Generated: in train: 14%, assess: 2%, novel: 84%
	 r oʊ ˈ g oʊ dʒ ɪ ŋ
	 ˈ d ɔ n ɝ
	 ˈ k æ d ə m ə s
	 ˈ h ɔ k n ɪ ŋ
	 ˈ l t ɑː n z
Epoch 212: train loss: 0.6037	dev loss: 0.8797                                                                                               
	Generated: in train: 9%, assess: 0%, n

Early stopping because of no decrease in 3 epochs.


In [ ]:
%%time


param_grid = ParameterGrid({
    'embedding_dimension': [10, 50, 100],
    'hidden_dimension': [10, 50, 100],
})

records = []
for params in tqdm(param_grid):
    model_parameters = ModelParams(rnn_type='gru', num_layers=1, max_epochs=1000, early_stopping_rounds=1, **params)
    model = LanguageModel(vocab, model_parameters, device_name='cuda')

    print('Model Params:', model_parameters)
    
    train_losses, dev_losses = model.fit(
        train_pronunciations.pronunciation.values.tolist(),
        dev_pronunciations.pronunciation.values.tolist()
    )
    
    for epoch, (train_loss, dev_loss) in enumerate(zip(train_losses, dev_losses), start=1):
        record = params.copy()
        record['epoch'] = epoch
        record['train_loss'] = train_loss
        record['dev_loss'] = dev_loss
        
        record['rnn_type'] = 'rnn'
        record['num_layers'] = 1 
    
        records.append(record)

models_df = pd.DataFrame.from_records(records)

In [ ]:
%%time


param_grid = ParameterGrid({
    'embedding_dimension': [50],
    'hidden_dimension': [2],
})

records = []
for params in tqdm(param_grid):
    model_parameters = ModelParams(rnn_type='rnn', num_layers=1, max_epochs=1000, early_stopping_rounds=1, **params)
    model = LanguageModel(vocab, model_parameters, device_name='cpu')

    print('Model Params:', model_parameters)
    
    train_losses, dev_losses = model.fit(
        train_pronunciations.pronunciation.values.tolist(),
        dev_pronunciations.pronunciation.values.tolist()
    )
    
    for epoch, (train_loss, dev_loss) in enumerate(zip(train_losses, dev_losses), start=1):
        record = params.copy()
        record['epoch'] = epoch
        record['train_loss'] = train_loss
        record['dev_loss'] = dev_loss
    
        records.append(record)

models_df = pd.DataFrame.from_records(records)

In [ ]:
models_df = pd.DataFrame.from_records(records)

There are {meow} models with a dev error of around .76. I'll choose the simplest one, which 

* point out that no matter how low the train error gets, the dev error
* which model parameters fail to ever get to the lowest dev error
* which model parameters overfit the most

* isolate the group of models with about .76 dev error. choose the simplest one.

In [ ]:
t = models_df[(models_df.embedding_dimension==50) & (models_df.hidden_dimension==100) & (models_df.num_layers==3)]
t = t.set_index('epoch')
t.dev_loss.plot()
t.train_loss.plot()
_ = plt.legend()

In [ ]:
models_df.sort_values('dev_loss').iloc[0]

Since the best "model" was at an earlier epoch I don't have access to it. So I'll train a model with that model's parameters and set the number of epochs to MEOW.

In [ ]:
%%time

model = PhonemeLM(
    phoneme_to_idx, device='cuda', rnn_type='gru',
    embedding_dimension=50, hidden_dimension=100, num_layers=3,
    max_epochs=69, early_stopping_rounds=69, batch_size=1024,
)

train_loss, dev_loss = model.fit(train_df.pronunciation.values.tolist(), dev_df.pronunciation.values.tolist())

Now that I have what I hope is the best model I can test how well it does on the holdout test set, which I haven't look at at all during the training process.

In [ ]:
test_loader = build_data_loader(test_df.pronunciation.values.tolist(), lm.phoneme_to_idx)
lm.evaluate(test_loader)

In [ ]:
dev_loader = build_data_loader(dev_df.pronunciation.values.tolist(), lm.phoneme_to_idx)
lm.evaluate(dev_loader)

TODO: compute the test error for the final model. Bar chart for the train, dev, and test errors

Comment on findings, probably that test is higher and that's expected because language models are very sensitive to corpus difference (and I probably overfit the dev set)

## Training the Final Model ##
Now that I've found the best parameters for the model according to the dev set I'll train a final model using all of the data. This should increase model performance overall since more data is better, but is also necessary since I'll be using the model to predict probabilities of all English words. If some of those words weren't in the training set they would artificially get lower probabilities. (Another approach here could be to train a model on e.g. 4/5 folds of the data and make predictions about the remaining 1/5, doing that 5 times to get unbiased predictions for all data, but this would have taken much longer to run.)

In [ ]:
%%time

model_params = ModelParams(
    rnn_type='gru', embedding_dimension=50, hidden_dimension=50, num_layers=1,
    max_epochs=3, early_stopping_rounds=3, batch_size=1024
)

language_model = LanguageModel(vocab, model_params, device_name='cpu')

_ = language_model.fit(pronunciations.pronunciation.values)

And finally, I'll save the model so I can use it in the next notebook, `Phoneme Exploration.ipynb`.

In [ ]:
with open('phoneme_language_model.pt', 'wb') as fh:
    model.save(fh)

In [ ]:
pd